# SVMs Entrenamiento

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.svm import NuSVR
from sklearn import metrics
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
# Para ignorar los warnings
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

In [3]:
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip", sep=",")

sotavento_columns = [col for col in wind_ava.columns if ".13" in col]
sotavento_columns.insert(0, "datetime")
sotavento_columns.append("energy")
wind_ava_sotavento = wind_ava[sotavento_columns]

wind_ava_sotavento = wind_ava_sotavento.rename(columns = {
    "energy": "Energy", 
    "datetime": "Datetime", 
    "p54.162.13": "Vertical integral of temperature", 
    "p55.162.13": "Vertical integral of water vapour", 
    "cape.13": "Convective available potential energy", 
    "p59.162.13": "Vertical integral of divergence of kinetic energy", 
    "lai_lv.13": "Leaf area index, low vegetation", 
    "lai_hv.13": "Leaf area index, high vegetation", 
    "u10n.13": "Neutral wind at 10 m u-component", 
    "v10n.13": "Neutral wind at 10 m v-component", 
    "sp.13": "Surface pressure",
    "stl1.13": "Soil temperature level 1",
    "u10.13": "10 metre U wind component",
    "v10.13": "10 metre V wind component",
    "t2m.13": "2 metre temperature", 
    "stl2.13": "Soil temperature level 2", 
    "stl3.13": "Soil temperature level 3", 
    "iews.13": "Instantaneous eastward turbulent surface stress",
    "inss.13": "Instantaneous northward turbulent surface", 
    "stl4.13": "Soil temperature level 4", 
    "fsr.13": "Forecast surface roughness", 
    "flsr.13": "Forecast logarithm of surface roughness for heat",
    "u100.13": "100 metre U wind component",
    "v100.13": "100 metre V wind component", })

wind_ava_sotavento["Datetime"] = pd.to_datetime(wind_ava_sotavento["Datetime"])
wind_ava_sotavento.dtypes

Datetime                                             datetime64[ns]
Vertical integral of temperature                            float64
Vertical integral of water vapour                           float64
Convective available potential energy                       float64
Vertical integral of divergence of kinetic energy           float64
Leaf area index, low vegetation                             float64
Leaf area index, high vegetation                            float64
Neutral wind at 10 m u-component                            float64
Neutral wind at 10 m v-component                            float64
Surface pressure                                            float64
Soil temperature level 1                                    float64
10 metre U wind component                                   float64
10 metre V wind component                                   float64
2 metre temperature                                         float64
Soil temperature level 2                        

In [4]:
duplicate_rows = wind_ava_sotavento[wind_ava_sotavento.duplicated()]
print("Número de filas duplicadas: ", duplicate_rows.shape)
null_rows = wind_ava_sotavento[wind_ava_sotavento.isnull().any(axis=1)]
print("Número de filas nulas: ", null_rows.shape)

Número de filas duplicadas:  (0, 24)
Número de filas nulas:  (0, 24)


In [5]:
import copy

# Dividir la columna 'Datetime' en dos nuevas columnas: 'fecha' y 'año'
wind_ava_sotavento['Month'] = wind_ava_sotavento['Datetime'].dt.month
wind_ava_sotavento['Year'] = wind_ava_sotavento['Datetime'].dt.year

# Eliminar la columna 'Datetime'
wind_ava_sotavento.drop(columns=['Datetime'], inplace=True)
# Ordenar el DataFrame wind_ava_sotavento por 'Datetime'
wind_ava_sotavento.sort_values(by=['Year', 'Month'], inplace=True)
# Crear una copia profunda del DataFrame wind_ava_sotavento
dataset_1 = copy.deepcopy(wind_ava_sotavento)
# Eliminar la columna 'Energy' de dataset_1
del dataset_1['Energy']
# Crear una copia profunda del DataFrame wind_ava_sotavento
dataset_3 = copy.deepcopy(wind_ava_sotavento)
dataset_2 = dataset_3["Energy"]
dataset_1.info()
dataset_2.info()

x_train, x_test, y_train, y_test = train_test_split(dataset_1, dataset_2, test_size=0.33, random_state=42)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4748 entries, 0 to 4747
Data columns (total 24 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Vertical integral of temperature                   4748 non-null   float64
 1   Vertical integral of water vapour                  4748 non-null   float64
 2   Convective available potential energy              4748 non-null   float64
 3   Vertical integral of divergence of kinetic energy  4748 non-null   float64
 4   Leaf area index, low vegetation                    4748 non-null   float64
 5   Leaf area index, high vegetation                   4748 non-null   float64
 6   Neutral wind at 10 m u-component                   4748 non-null   float64
 7   Neutral wind at 10 m v-component                   4748 non-null   float64
 8   Surface pressure                                   4748 non-null   float64
 9   Soil tem

# SVM: Evaluación con hiperparámetros por omisión

In [7]:
# Creamos la clase de SVR
t_inicio = time.time() # Guardamos el tiempo inicial
svr = SVR()
pipe = Pipeline([('SVM', svr)])

pipe.fit(x_train, y_train) # Se entrena el modelo
accuracy_train_svr_og = pipe.score(x_test, y_test)  # Se obtiene el resultado de aciertos
t_fin = time.time() # Guardamos el tiempo final
print('El modelo SVR con hiperparámetros por omisión ha obtenido una media de resultado de {:.12f}'.format(accuracy_train_svr_og))
# Miramos sus errores
prediction = pipe.predict(x_test)
error_rmse_svr_og = sqrt(mean_squared_error(y_test, prediction))
error_mae_svr_og = mean_absolute_error(y_test, prediction)
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_svr_og, error_mae_svr_og))
t_svr_og = t_fin - t_inicio
print('El tiempo de ejecucion ha sido de {}'.format(t_svr_og))

El modelo SVR con hiperparámetros por omisión ha obtenido una media de resultado de 0.047713612563
Como errores:
	RMSE	= 648.1658531998673
	MAE	= 488.9170329051875
El tiempo de ejecucion ha sido de 0.49689722061157227


LinearSVR

In [9]:
# Creamos la clase de LinearSVR
t_inicio = time.time() # Guardamos el tiempo inicial
svr = LinearSVR()

svr.fit(x_train,y_train) # Se entrena el modelo
accuracy_train_lsvr_og = svr.score(x_test, y_test)  # Se obtiene el resultado de aciertos
t_fin = time.time() # Guardamos el tiempo final
print('El modelo LinearSVR con hiperparámetros por omisión ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_lsvr_og))
# Miramos sus errores
prediction = svr.predict(x_test)
error_rmse_lsvr_og = sqrt(mean_squared_error(y_test, prediction))
error_mae_lsvr_og = mean_absolute_error(y_test, prediction)
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_lsvr_og, error_mae_lsvr_og))
t_lsvr_og = t_fin - t_inicio
print('El tiempo de ejecucion ha sido de {}'.format(t_lsvr_og))

El modelo LinearSVR con hiperparámetros por omisión ha obtenido una media de resultado de -0.18
Como errores:
	RMSE	= 720.0990725993651
	MAE	= 514.4958279030099
El tiempo de ejecucion ha sido de 0.16043710708618164


c:\Users\aitan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


NuSVR

In [10]:
t_inicio = time.time() # Guardamos el tiempo inicial
svr = NuSVR()
np.random.seed(42)  # Para la toma de decisión entre varios puntos
svr.fit(x_train,y_train) # Se entrena el modelo
accuracy_train_nusvr_og = svr.score(x_test, y_test)  # Se obtiene el resultado de aciertos
t_fin = time.time() # Guardamos el tiempo final
print('El modelo NuSVR con hiperparámetros por omisión ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_nusvr_og))
# Miramos sus errores
prediction = svr.predict(x_test)
error_rmse_nusvr_og = sqrt(mean_squared_error(y_test, prediction))
error_mae_nusvr_og = mean_absolute_error(y_test, prediction)
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_nusvr_og, error_mae_nusvr_og))
t_nusvr_og = t_fin - t_inicio
print('El tiempo de ejecucion ha sido de {}'.format(t_nusvr_og))

El modelo NuSVR con hiperparámetros por omisión ha obtenido una media de resultado de 0.10
Como errores:
	RMSE	= 628.7370541413912
	MAE	= 506.96909680449835
El tiempo de ejecucion ha sido de 0.2984771728515625


Resultados:

In [11]:
print('El modelo SVR con hiperparámetros por omisión ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_svr_og))
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_svr_og, error_mae_svr_og))
print('El tiempo de ejecucion ha sido de {}'.format(t_svr_og))
print("")
print('El modelo LinearSVR con hiperparámetros por omisión ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_lsvr_og))
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_lsvr_og, error_mae_lsvr_og))
print('El tiempo de ejecucion ha sido de {}'.format(t_lsvr_og))
print("")
print('El modelo NuSVR con hiperparámetros por omisión ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_nusvr_og))
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_nusvr_og, error_mae_nusvr_og))
print('El tiempo de ejecucion ha sido de {}'.format(t_nusvr_og))

El modelo SVR con hiperparámetros por omisión ha obtenido una media de resultado de 0.05
Como errores:
	RMSE	= 648.1658531998673
	MAE	= 488.9170329051875
El tiempo de ejecucion ha sido de 0.49689722061157227

El modelo LinearSVR con hiperparámetros por omisión ha obtenido una media de resultado de -0.18
Como errores:
	RMSE	= 720.0990725993651
	MAE	= 514.4958279030099
El tiempo de ejecucion ha sido de 0.16043710708618164

El modelo NuSVR con hiperparámetros por omisión ha obtenido una media de resultado de 0.10
Como errores:
	RMSE	= 628.7370541413912
	MAE	= 506.96909680449835
El tiempo de ejecucion ha sido de 0.2984771728515625
